## DImensionality reduction and manifold learning (embeddings)
* Today we are going to learn how to visualize and explore the data
![pictcha](http://sarahannelawless.com/wp-content/uploads/2015/03/tw-1-600x449.jpg)

In [ ]:
from time import time
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

### Load digits
 * A thousand something 8x8 handwritten digits

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape


In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# a few testimonials
plt.subplot(1,2,1)
plt.imshow(X[0].reshape(8,8))
plt.subplot(1,2,2)
plt.imshow(X[1].reshape(8,8))

### Data visualization

Here's an overly complicated function that draws any two-dimensional space you want

In [ ]:
from matplotlib import offsetbox
def plot_embedding(X,y=None,ax=None,show_images=True,min_dist=5e-3,figsize=[12,10]):
    
    #normalize
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    if ax is None:
        plt.figure(figsize=figsize)
        ax = plt.subplot(1,1,1)

    # scatter-plot
    if y is None:
        plt.scatter(*X.T)
    else:
        assert y is not None
        #рисуем циферки a-la scatter
        for i in range(X.shape[0]):
            ax.text(X[i, 0], X[i, 1], str(y[i]),
                     color= plt.cm.Set1(y[i] / 10.),
                     fontdict={'weight': 'bold', 'size': 9})

    if not show_images:
        return
        
    shown_images = np.array([[1., 1.]])  # just something big
    for i in range(X.shape[0]):
        dist = np.sum((X[i] - shown_images) ** 2, 1)
        if np.min(dist) < min_dist: continue
        shown_images = np.r_[shown_images, [X[i]]]
        imagebox = offsetbox.AnnotationBbox(
            offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
            X[i])
        ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])


### GaussianRandomProjection
 * Pick several random axes from normal distribution
 * Projects data to these axis
 * Mostly useless for our task.

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

Xrp = GaussianRandomProjection(n_components=2).fit_transform(X)

In [ ]:
plot_embedding(Xrp)

### Wut?!
Not super helpful to say the least.

In [ ]:
plot_embedding(Xrp,y)

Random projections will change if you re-run code.

### Singular Value Decomposition

* Idea: try to compress the data in a way that you can then restore it
* Equivalent to minimizing MSE: $|| X  - U \cdot \Sigma \cdot V^T ||$

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2)
Xsvd = svd.fit_transform(X)

In [ ]:
plot_embedding(Xsvd[:,:2],y)

In [ ]:
n = svd.n_components
plt.figure(figsize=[16,8])
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.imshow(svd.components_[i].reshape(8,8),
              cmap='gray',interpolation='none')
    plt.colorbar()


### PCA aka Principial Component Analysis
* Idea: we try to find axes that "explain as much variance as possible"
* Mathematically it's almost the same as SVD
* But we'll make you train it anyway

In [ ]:
from sklearn.decomposition import PCA
pca = <create it>

Xpca = <transform data>

In [ ]:
plot_embedding(Xpca,y)

#### Let's not compare all 3...

In [ ]:
plt.figure(figsize=[12,4])
plot_embedding(Xrp,y,ax=plt.subplot(1,3,1),min_dist=3e-2)
plot_embedding(Xsvd,y,ax=plt.subplot(1,3,2),min_dist=3e-2)
plot_embedding(Xpca,y,ax=plt.subplot(1,3,3),min_dist=3e-2)

# LDA aka Linear Discriminant Analysis

* Idea: let's find axes that allow us to better separate signal from background.
* Unlike PCA, this guy actually needs Y data. So you'll have to fit(X,y).


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = <create>
Xlda = <fit_transform>

In [ ]:
plot_embedding(Xlda,y)

#### We won!

##### Okay, time to make things a bit more complex

__Scroll back to the first tab, find the load_digits function and change n_class to 10__

Then re-run everything.

```

```

```

```

```

```

```

```

```

```

```

```

```

```



# Embedding aka Manifold learning

* Instead of finding linear axes, try to just every image with 2D point in a dictionary fashion.

* General idea: assign images with 2d points to minimize some structural error. Each algorithm uses it's own error metric.

### Multidimensional Scaling

* Idea: assign similar images into close points and keep different images at distance.

#####  More complicated
* We try to preserve euclidian distance
  * Images can be represented as pixel vectors with euclidian distance defined as
$$ r(a,b) = \sqrt { (a_0 - b_0)^2 + (a_1 - b_1)^2 + ... + (a_63 - b_63)^2}$$
  * Our new points are two-dimensional, with 2d euclidian distance
$$ r_{new}(a',b') = \sqrt { (a'_x - b'_x)^2 + (a'_y - b'_y)^2 } $$

  * We want $r_{new}$ to be close to $r$
  * Move $a'$ and $b'$ to minimize MSE:
$$ \sum _{a,b,a',b'} (r_{new}(a',b') - r(a,b))^2 \to min_{a',b'} $$

  * In code, the mean squared error is denoted as  __stress__

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components=2,verbose=2,n_init=1)
Xmds = mds.fit_transform(X)

In [ ]:
plot_embedding(Xmds,y)

# t-SNE
t-distributed Stochasitc Neiborhood Embedding

Ideologically similar to MDS, but with two differences:
* It only cares about close points. The distant points aren't as important.
* It uses a different notion of distance (like geodesic distance in graph) formulated through likelihood.


In short, it tries to preserve local structure, giving up on global structure.

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
%%time
tsne = <create>
Xtsne = <fit_transform>

In [ ]:
plot_embedding(Xtsne,y)

# Tuning it
 * t-SNE зависит depends on perplexity ~ how many neighbors to take in account
   * See how result changing if you scale perplexity between 1 and 100


In [ ]:
<Try 5 different perplexities>

 * Попробуйте сперва выделить несколько главных компонент с PCA, и только потом применить TSNE
   * Для начала, 64D изначально -> 16D после PCA -> 2D после TSNE


In [ ]:
<PCA_or_LDA->tsne>

There's a lot more to manifold learning.

* Sklearn page - http://scikit-learn.org/stable/modules/manifold.html
* Interactive demo - http://distill.pub/2016/misread-tsne/